In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
from lxml import html

df = pd.read_csv('data_linkjob_test.csv')
# get links
links = df['link']
# create field 
df['job_name'] = ''
df['id_company'] = ''
df['company_name'] = ''
df['job_salary'] = ''
df['job_experience'] = ''
df['job_level'] = ''
df['job_expired_date'] = ''
df['job_details'] = ''
df['job_required'] = ''

In [4]:
def get_job_name(soup):
    try:
        div_tag = soup.find('div', {'class': 'job-desc'})
        tag_h1 = div_tag.find('h1', {'class': 'title'})
        if tag_h1: 
            job_name = tag_h1.get_text()
        else:
            tag_h2 = soup.find('div', {'class': 'title'})
            job_name = tag_h2.find('h2').get_text()
    except : return None
    return job_name

In [53]:
def get_job_id_company(soup):
    try:
        div_tag = soup.find('div', {'class': 'job-desc'})
        div_tag2 = div_tag.find('a', {'class': 'employer job-company-name'})
        if div_tag2:
            href = div_tag2['href']
            job_id_company = href.split('.')[-2]
        else: job_id_company = ''
    except : return None
    return job_id_company
    

In [54]:
def get_job_company_name(soup):
    try:
        div_tag = soup.find('div', {'class': 'job-desc'})
        a_tag = div_tag.find('a', {'class': 'employer job-company-name'})
        if a_tag:
            job_company = a_tag.get_text()
        else: job_company = ''
    except : return None
    return job_company

In [55]:
def get_job_salary(soup):
    try: 
        i_tag = soup.find('i', {'class': 'fa fa-usd'})
        li_tag = i_tag.parent.parent
        salary = li_tag.find('p').get_text()
    except: return None
              
    return salary

In [56]:
def get_job_experience(soup):
    try: 
        i_tag = soup.find('i', {'class': 'fa fa-briefcase'})
        if i_tag :
            li_tag = i_tag.parent.parent
            experience = li_tag.find('p').get_text().strip()
            clean = re.sub(r'\s+', ' ', experience)
            experience = clean
        else :experience = "Không yêu cầu kinh nghiệm"
    except : return None
    
    return experience

In [57]:
def get_job_level(soup):
    try:
        i_tag = soup.find('i', {'class': 'mdi mdi-account'})
        if i_tag :
            li_tag = i_tag.parent.parent
            level = li_tag.find('p').get_text().strip()
        else :level = None
    except : return None
    return level

In [58]:
def get_job_expired_date(soup):
    try:
        i_tag = soup.find('i', {'class': 'mdi mdi-calendar-check'})
        if i_tag :
            li_tag = i_tag.parent.parent
            expired_date = li_tag.find('p').get_text().strip()
        else :expired_date = None
    except : return None
    
    return expired_date

In [59]:
def get_job_details(soup):
    try:
        div_tag = soup.find('div', {'class': 'detail-row reset-bullet'})
        # p_tags = div_tag.find_all('p')
        # job_details = ""
        # for p_tag in p_tags :
        #     job_details = job_details + "/n" + p_tag.get_text()
        job_details = div_tag.get_text(separator='\n', strip=True)
    except : return None
    return job_details

In [60]:
def get_job_required(soup):
    try: 
        selected_element = soup.select_one('#tab-1 > section > div:nth-child(4)')
        job_required = selected_element.get_text(separator='\n', strip=True)
    except : return None
    return job_required

In [ ]:
# Khởi tạo trình duyệt Edge
driver = webdriver.Edge()
i = 0
# Duyệt qua từng link và lấy thông tin
for link in links:
    # Mở trang web
    driver.get(link)
    # Chờ cho trang web tải xong
    driver.implicitly_wait(10)
    # Lấy HTML của trang web
    html = driver.page_source
    # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    #find job name
    job_name = get_job_name(soup)
    df['job_name'][i] = job_name
    #find company id
    id_company = get_job_id_company(soup)
    df['id_company'][i] = id_company
    #find company name
    company_name = get_job_company_name(soup)
    df['company_name'][i] = company_name
    #find job salary 
    job_salary = get_job_salary(soup)
    df['job_salary'][i] = job_salary
    #find job experience
    job_experience = get_job_experience(soup)
    df['job_experience'][i] = job_experience
    #find job level
    job_level = get_job_level(soup)
    df['job_level'][i] = job_level
    #find job expired date
    job_expired_date = get_job_expired_date(soup)
    df['job_expired_date'][i] = job_expired_date
    #find job details
    job_details = get_job_details(soup)
    df['job_details'][i] = job_details
    #find job required information
    job_required = get_job_required(soup)
    df['job_required'][i] = job_required
    print(str(i)+ " done")
    i = i + 1
    df.to_csv('data_done_3.csv', index=False)

# Đóng trình duyệt
driver.quit()


# Convert CSV to JSON

In [5]:
import json
# Đọc dữ liệu từ file CSV hoặc thực hiện các bước để có DataFrame của bạn
dfr = pd.read_csv('job_data.csv')
# Chuyển đổi DataFrame thành danh sách bản ghi
records = dfr.to_dict(orient='records')

# Chuyển đổi dữ liệu ngày thành định dạng chuẩn (ví dụ: ISO 8601)
for record in records:
    if 'job_name' in record and pd.isna(record['job_name']):
        record['job_name'] = None
    if 'job_expired_date' in record and pd.notna(record['job_expired_date']):
        record['job_expired_date'] = pd.to_datetime(record['job_expired_date']).isoformat()
    if 'job_experience' in record and pd.isna(record['job_experience']):
        record['job_experience'] = None
    if 'id_company' in record and pd.isna(record['id_company']):
        record['id_company'] = None
    if 'company_name' in record and pd.isna(record['company_name']):
        record['company_name'] = None
    if 'job_salary' in record and pd.isna(record['job_salary']):
        record['job_salary'] = None
    if 'job_experience' in record and pd.isna(record['job_experience']):
        record['job_experience'] = None
    if 'job_level' in record and pd.isna(record['job_level']):
        record['job_level'] = None
    if 'job_expired_date' in record and pd.isna(record['job_expired_date']):
        record['job_expired_date'] = None
    if 'job_details' in record and pd.isna(record['job_details']):
        record['job_details'] = None
    if 'job_required' in record and pd.isna(record['job_required']):
        record['job_required'] = None
    if 'company_logo' in record and pd.isna(record['company_logo']):
        record['company_logo'] = None
    if 'job_address' in record and pd.isna(record['job_address']):
        record['job_address'] = None
# Ghi vào file JSON mà không sử dụng Unicode Escape
with open('job_data_final.json', 'w', encoding='utf-8') as json_file:
    json.dump(records, json_file, ensure_ascii=False, indent=2)
# Lưu DataFrame thành file JSON
#dfr.to_json('test.json', orient='records', lines=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5092\330429528.py:12: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  record['job_expired_date'] = pd.to_datetime(record['job_expired_date']).isoformat()


In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Thông tin kết nối đến cơ sở dữ liệu MySQL
db_username = 'root'
db_password = ''
db_host = '127.0.0.1'
db_name = 'php_resful_api'

# Tạo đối tượng kết nối đến cơ sở dữ liệu
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

# Đọc dữ liệu từ file CSV
csv_file_path = './data/job_data_final.csv'
df = pd.read_csv(csv_file_path)
# Thêm cột ID tự tăng
df.insert(0, 'id', range(1, 1 + len(df)))
print(df)
# Loại bỏ dòng có dữ liệu trống
df = df.dropna()
# Xóa cột 'link' từ DataFrame trước khi lưu vào cơ sở dữ liệu
df = df.drop(columns=['link'])

# Lưu dữ liệu vào cơ sở dữ liệu MySQL
table_name = 'data'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)


        id                                               link  \
0        1  https://careerviet.vn/vi/tim-viec-lam/dai-dien...   
1        2  https://careerviet.vn/vi/tim-viec-lam/chuyen-v...   
2        3  https://careerviet.vn/vi/tim-viec-lam/nhan-vie...   
3        4  https://careerviet.vn/vi/tim-viec-lam/truong-p...   
4        5  https://careerviet.vn/vi/tim-viec-lam/it-purch...   
...    ...                                                ...   
5270  5271  https://careerviet.vn/vi/tim-viec-lam/3d-cad-o...   
5271  5272  https://careerviet.vn/vi/tim-viec-lam/ke-toan-...   
5272  5273  https://careerviet.vn/vi/tim-viec-lam/payment-...   
5273  5274  https://careerviet.vn/vi/tim-viec-lam/innovati...   
5274  5275  https://careerviet.vn/vi/tim-viec-lam/nhan-vie...   

                                               job_name id_company  \
0              Đại diện bán hàng – Sales Representative   35A891E7   
1                    CHUYÊN VIÊN MUA HÀNG (Tiếng Trung)   35A67947   
2        

4927